https://colab.research.google.com/drive/1mq6dEJ4O6YK92DnpxlKlcW50PgaMdc4P?usp=sharing  CLICK HERE TO VIEW ALL OUTPUTS!!! - Nolan 

In [ ]:
!pip install scikeras

In [ ]:
from tensorflow.keras import backend as K
# import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.random import set_seed
from random import seed
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Infor648/Data/Data3.csv'
df = pd.read_csv(file_path, encoding='cp1252')  # Windows-1252
print(f"Loaded {len(df)} entries")
print(df.head())

In [ ]:
df_sub = df[[
    "Data"
]]

# Display the first few rows of the subset DataFrame
df_sub.head()

In [ ]:
import re
import pandas as pd

# Combine all data entries into one string
combined_text = ' '.join(df_sub['Data'])

# Tokenize combined text
language = []
words = re.split(r'[ ]\s*', combined_text)
language.extend(words)

# Dictionary to hold unique words and their assigned numbers
tokens = {}
counter = 1

tokens["."] = 377

# Assign tokens to each unique word
for word in language:
    if word not in tokens:
        tokens[word] = counter
        counter += 1

# Function to convert text to tokens, splitting periods
def phrase_converter(text, tokens):
    # Add space before periods to match initial tokenization
    text = re.sub(r'(\w)\.', r'\1 .', text)
    words = re.split(r'[ ]\s*', text)
    return ' '.join(str(tokens.get(word, 0)) for word in words)

# Apply the conversion to each entry
df_sub['sequence'] = df_sub['Data'].apply(lambda x: phrase_converter(x, tokens))


# Print token encoding summary
print("Token Encoding:")
print(df_sub['sequence'].value_counts())

# Function to create [word, token] pairs for each sequence
def create_word_token_pairs(sequence, token_dict):  # Renamed to avoid confusion
    # Reverse the tokens dictionary for lookup
    reverse_tokens = {v: k for k, v in token_dict.items()}  # token_dict should be a dict here
    # Split the sequence into token IDs
    token_ids = sequence.split()
    # Create [word, token] pairs
    pairs = [[reverse_tokens.get(int(token), "UNKNOWN"), int(token)] for token in token_ids]
    return pairs

# Add the [word, token] pairs to a new column
df_sub['dictionary'] = df_sub['sequence'].apply(lambda x: create_word_token_pairs(x, tokens))

# Print the updated DataFrame
print("\nUpdated DataFrame with 'dictionary':")
print(df_sub[['Data', 'sequence', 'dictionary']])

In [ ]:
print("Washington token:", tokens.get("Washington"))

In [ ]:
df_sub.head()

In [ ]:
# Apply the conversion to each entry
df_sub['sequence'] = df_sub['Data'].apply(lambda x: phrase_converter(x, tokens))

# Prepare X and y with a window size of 2
X = []
y = []
window_size = 7

for sequence in df_sub['sequence']:
    tokens = sequence.split()  # Split sequence into tokens
    tokens = [int(token) for token in tokens]  # Convert tokens to integers
    # Ensure there are enough tokens for window_size + 1 (2 inputs + 1 output)
    for i in range(len(tokens) - window_size):
        X.append(tokens[i:i + window_size])  # Two consecutive tokens
        y.append(tokens[i + window_size])    # Next token

X = np.array(X)  # Shape: (num_samples, 2)
y = np.array(y)  # Shape: (num_samples,)

In [ ]:
#Split the dataset into training and testing sets test_size using 0.3: 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#Display the shapes of the resulting datasets
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)

In [ ]:
X_train

Nueral Network


In [ ]:
y_train

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Activation Functions (unchanged)
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

# SimpleNeuralNetwork class (unchanged)
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size=128, output_size=377, learning_rate=0.01):
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros((1, output_size))
        self.lr = learning_rate
        self.loss_history = []

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = softmax(self.z2)
        return self.a2

    def backward(self, X, y, output):
        n_samples = X.shape[0]
        y_adjusted = y - 1
        y_one_hot = np.zeros((n_samples, self.W2.shape[1]))
        y_one_hot[np.arange(n_samples), y_adjusted] = 1

        self.error = output - y_one_hot
        self.delta2 = self.error

        self.delta1 = np.dot(self.delta2, self.W2.T) * relu_derivative(self.a1)

        self.dW2 = np.dot(self.a1.T, self.delta2) / n_samples + 0.01 * self.W2
        self.db2 = np.sum(self.delta2, axis=0, keepdims=True) / n_samples
        self.dW1 = np.dot(X.T, self.delta1) / n_samples + 0.01 * self.W1
        self.db1 = np.sum(self.delta1, axis=0, keepdims=True) / n_samples

        self.W2 -= self.lr * self.dW2
        self.b2 -= self.lr * self.db2
        self.W1 -= self.lr * self.dW1
        self.b1 -= self.lr * self.db1

    def train(self, X, y, epochs=100, batch_size=32):
        for epoch in range(epochs):
            indices = np.random.permutation(X.shape[0])
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            for i in range(0, X.shape[0], batch_size):
                batch_X = X_shuffled[i:i + batch_size]
                batch_y = y_shuffled[i:i + batch_size]
                output = self.forward(batch_X)
                self.backward(batch_X, batch_y, output)

            output_full = self.forward(X)
            loss = -np.mean(np.log(output_full[np.arange(X.shape[0]), y - 1] + 1e-10))
            self.loss_history.append(loss)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1) + 1

    def plot_loss(self):
        plt.plot(range(1, len(self.loss_history) + 1), self.loss_history, label='Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss Over Epochs')
        plt.legend()
        plt.grid(True)
        plt.show()

# Your data prep (unchanged)
# Pad sequences with [0, 0, 1], [0, 1, 2], etc.
df_sub['sequence'] = [f'{"0 " * (window_size - 1)}{seq}' for seq in df_sub['sequence']]

# Prepare X and y with window size 3
X = []
y = []
window_size = 7
for sequence in df_sub['sequence']:
    tokens = sequence.split()
    tokens = [int(token) for token in tokens]
    for i in range(len(tokens) - window_size):
        X.append(tokens[i:i + window_size])
        y.append(tokens[i + window_size])

X = np.array(X)
y = np.array(y)

# One-hot encode X
vocab_size = 377
num_samples = X.shape[0]
X_one_hot = np.zeros((num_samples, window_size * vocab_size))
for i in range(window_size):
    X_one_hot[np.arange(num_samples), (i * vocab_size) + X[:, i] - 1] = 1

# Train the model
nn = SimpleNeuralNetwork(input_size=window_size * vocab_size, hidden_size=64, output_size=377, learning_rate=0.1)
nn.train(X_one_hot, y, epochs=100, batch_size=32)

# Plot the loss
nn.plot_loss()

# Generation loop with last_prediction penalty
max_tokens = 20
end_token = 377  # "." is 377
initial_tokens = [1, 2, 3, 4, 5, 6, 7]  # George Washington ?
generated_tokens = initial_tokens.copy()
current_window = initial_tokens.copy()
last_prediction = None

for _ in range(max_tokens):
    test_input = np.zeros((1, window_size * vocab_size))
    for i, token in enumerate(current_window):
        test_input[0, i * vocab_size + token - 1] = 1
    probs = nn.forward(test_input)  # Get raw probabilities
    # Penalize 158 if it was the last prediction
    if last_prediction == end_token:
        probs[0, end_token - 1] *= 0.5  # Reduce probability of 158 by 90%
    prediction = np.argmax(probs, axis=1)[0] + 1  # Pick highest after penalty
    generated_tokens.append(prediction)
    current_window.pop(0)
    current_window.append(prediction)
    last_prediction = prediction  # Update last prediction
    if prediction == end_token:
        break
# Function to extract reverse tokens from df_sub['dictionary']
def get_reverse_tokens_from_dictionary(df_dictionary):
    reverse_tokens = {}
    for sequence_pairs in df_dictionary:
        for word, token in sequence_pairs:
            reverse_tokens[token] = word  # Map token ID to word
    return reverse_tokens

# Function to convert generated tokens to words using df_sub['dictionary'] (unchanged)
def tokens_to_words_from_dict(generated_tokens, df_dictionary):
    if isinstance(generated_tokens, list):
        token_str = ' '.join(map(str, generated_tokens))
    else:
        token_str = generated_tokens
    token_ids = token_str.split()
    reverse_tokens = get_reverse_tokens_from_dictionary(df_dictionary)
    return ' '.join(reverse_tokens.get(int(token), "UNKNOWN") for token in token_ids)

# Convert to words (assuming your token-to-word function is defined)
word_sequence = tokens_to_words_from_dict(generated_tokens, df_sub['dictionary'])
print(f"Generated sequence: {generated_tokens}")
print(f"Word sequence: {word_sequence}")

# Your existing debug code (unchanged)
probs = nn.forward(test_input)
top_5_indices = np.argsort(probs[0])[::-1][:5] + 1
top_5_probs = probs[0, top_5_indices - 1]
print("Top 5 predicted tokens and probabilities:")
for idx, prob in zip(top_5_indices, top_5_probs):
    print(f"Token {idx}: {prob:.4f}")

from collections import Counter
targets = []
for i in range(num_samples):
    if np.array_equal(X[i], [1, 2, 3]):
        targets.append(y[i])
print("Tokens following [1, 2, 3]:", Counter(targets))

In [ ]:
# Parameters
window_size = 7
vocab_size = 377
max_tokens = 20
end_token = 376

# Initial sequence
initial_tokens = [1, 2, 3, 5, 6, 7]  # e.g., "George Washington ?"
generated_tokens = initial_tokens.copy()  # Start with initial tokens
current_window = initial_tokens.copy()
last_prediction = None  # Track the last predicted token

# Generation loop with repetition penalty
for _ in range(max_tokens):
    # Prepare one-hot encoded input
    test_input = np.zeros((1, window_size * vocab_size))
    for i, token in enumerate(current_window):
        test_input[0, i * vocab_size + token - 1] = 1  # One-hot encode

    # Get probabilities and apply penalty if last was 158
    probs = nn.forward(test_input)  # Shape: [1, 158]
    if last_prediction == 377:  # Penalize "." if it was last
        probs[0, 377 - 1] *= 0.001  # Reduce probability of 158 by 90%
    prediction = np.argmax(probs, axis=1)[0] + 1  # Get scalar prediction after penalty

    # Append to sequence
    generated_tokens.append(prediction)

    # Shift window: drop leftmost, add new prediction
    current_window.pop(0)
    current_window.append(prediction)

    # Update last prediction
    last_prediction = prediction

    # Stop if end_token is predicted
    if prediction == end_token:
        break

# Function to extract reverse tokens from df_sub['dictionary'] (unchanged)
def get_reverse_tokens_from_dictionary(df_dictionary):
    reverse_tokens = {}
    for sequence_pairs in df_dictionary:
        for word, token in sequence_pairs:
            reverse_tokens[token] = word  # Map token ID to word
    return reverse_tokens

# Function to convert generated tokens to words using df_sub['dictionary'] (unchanged)
def tokens_to_words_from_dict(generated_tokens, df_dictionary):
    if isinstance(generated_tokens, list):
        token_str = ' '.join(map(str, generated_tokens))
    else:
        token_str = generated_tokens
    token_ids = token_str.split()
    reverse_tokens = get_reverse_tokens_from_dictionary(df_dictionary)
    return ' '.join(reverse_tokens.get(int(token), "UNKNOWN") for token in token_ids)

# Convert the generated sequence to words
word_sequence = tokens_to_words_from_dict(generated_tokens, df_sub['dictionary'])

# Print results
print(f"Generated sequence: {generated_tokens}")
print(f"Word sequence: {word_sequence}")

In [ ]:
import numpy as np

# Parameters
window_size = 7
vocab_size = 377  # Total number of unique tokens
max_tokens = 20
end_token = 378  # Period token to stop generation

# Assume nn is your trained neural network model (e.g., an instance of your class)

# Function to extract reverse tokens from df_sub['dictionary']
def get_reverse_tokens_from_dictionary(df_dictionary):
    reverse_tokens = {}
    for sequence_pairs in df_dictionary:
        for word, token in sequence_pairs:
            reverse_tokens[token] = word  # Map token ID to word
    return reverse_tokens

# Function to convert tokens to words using df_sub['dictionary']
def tokens_to_words_from_dict(generated_tokens, df_dictionary):
    if isinstance(generated_tokens, list):
        token_str = ' '.join(map(str, generated_tokens))
    else:
        token_str = generated_tokens
    token_ids = token_str.split()
    reverse_tokens = get_reverse_tokens_from_dictionary(df_dictionary)
    return ' '.join(reverse_tokens.get(int(token), "UNKNOWN") for token in token_ids)

# Function to get token ID from a word using df_sub['dictionary']
def word_to_token(word, df_dictionary):
    reverse_tokens = get_reverse_tokens_from_dictionary(df_dictionary)
    # Create forward mapping: word -> token
    forward_tokens = {v: k for k, v in reverse_tokens.items()}
    return forward_tokens.get(word, 0)  # Return 0 if word not found

# Prompt user for input and generate sequence
def generate_sequence_from_word(nn, df_dictionary):
    # Get input from user
    input_word = input("Enter a word to start the sequence: ").strip()

    # Convert input word to token
    start_token = word_to_token(input_word, df_dictionary)
    if start_token == 0:
        print(f"Word '{input_word}' not found in dictionary. Using token 0.")

    # Initialize sequence with the single token (padded with zeros for window)
    initial_tokens = [0] * (window_size - 1) + [start_token]  # Pad with zeros to fill window
    generated_tokens = [start_token]  # Start sequence with the input token
    current_window = initial_tokens.copy()  # Full window for prediction
    last_prediction = None

    # Generation loop
    for _ in range(max_tokens):
        # Prepare one-hot encoded input
        test_input = np.zeros((1, window_size * vocab_size))
        for i, token in enumerate(current_window):
            test_input[0, i * vocab_size + token - 1] = 1  # One-hot encode (adjust for 1-based tokens)

        # Get probabilities and apply penalty if last was 377 (period)
        probs = nn.forward(test_input)  # Shape: [1, vocab_size]
        if last_prediction == 377:  # Penalize "." if it was last
            probs[0, 377 - 1] *= 0.0001  # Reduce probability of period by 99.9%
        prediction = np.argmax(probs, axis=1)[0] + 1  # Get scalar prediction (1-based)

                # Get probabilities and apply penalty if last was 377 (period)
        probs = nn.forward(test_input)  # Shape: [1, vocab_size]
        if last_prediction == 376:  # Penalize "." if it was last
            probs[0, 376 - 1] *= 0.001  # Reduce probability of period by 99.9%
        prediction = np.argmax(probs, axis=1)[0] + 1  # Get scalar prediction (1-based)

        # Append to sequence
        generated_tokens.append(prediction)

        # Shift window: drop leftmost, add new prediction
        current_window.pop(0)
        current_window.append(prediction)

        # Update last prediction
        last_prediction = prediction

        # Stop if end_token is predicted
        if prediction == end_token:
            break

    # Convert tokens to words
    word_sequence = tokens_to_words_from_dict(generated_tokens, df_dictionary)

    # Print results
    print(f"Generated sequence: {generated_tokens}")
    print(f"Word sequence: {word_sequence}")

# Example usage (assuming nn and df_sub are defined)
# Replace 'nn' with your actual neural network instance
generate_sequence_from_word(nn, df_sub['dictionary'])